In [9]:
import pandas as pd

def create_data_subjects(interval):

    filepath_main = '../data/'
    df = pd.read_csv(filepath_main + 'email headers.csv', encoding='cp1252')
    df.Date = pd.to_datetime(df.Date)
    df.head()

    dffrom = list(df.From)
    dfto = list(df.To)
    dftone = []
    for row in dfto:
        dftone.append(row.split(', '))

    dffrom2 = []
    for row in dffrom:
        if row.split('@')[1] == "gastech.com.tethys":
            dffrom2.append((row.split('@')[0].split('.')[0] + ' ' + row.split('@')[0].split('.')[1]) + ' (tethys)')
        else:
            dffrom2.append((row.split('@')[0].split('.')[0] + ' ' + row.split('@')[0].split('.')[1]))

    ef_to = []
    for row in dftone:
        l = []
        for name in row:
            if name.split('@')[1] == "gastech.com.tethys":
                l.append((name.split('@')[0].split('.')[0] + ' ' + name.split('@')[0].split('.')[1]) + ' (tethys)')
            l.append((name.split('@')[0].split('.')[0] + ' ' + name.split('@')[0].split('.')[1]))
        ef_to.append(l)
    df = pd.DataFrame({ 'from': dffrom2, 'to':ef_to, 'date': df.Date, 'subject':df.Subject})

    df['interval'] = df['date'].dt.round('{}min'.format((str(interval))))
    df.head()

    data = df.values.tolist()

    dicdat = {}
    for row in data:
        day = str(row[4])
        if day in dicdat.keys():
            dicdat[day].append(row)
        else:
            dicdat[day] = [row]


    dic = {}

    for key in dicdat.keys():
        lst = dicdat[key]
        lh = []
        for row in lst:
            source = row[0]
            for tat in row[1]:
                lh.append([source, tat, row[3]])
        dic[key] = lh

    l = []

    for k,v in dic.items():
        l.append([k,v])

    pd.DataFrame(l, columns=['Day', 'Network']).to_csv('../data/networkplot_data/network_data_{}_subjects.csv'.format(str(interval)))

def create_networkformat_perday(day:str, dicdat):
    data = dicdat[day]
    df = pd.DataFrame(data, columns = ['from','to','date','subject', 'day'])
    unique_froms = (set(list(df['from'])))
    
    dic = {}
    for i in range(len(df['from'])):
        k = df['from'][i]
        if k in dic.keys():
            dic[k] += df['to'][i]
        else:
            dic[k] = df['to'][i]

    l = []
    for name in unique_froms:
        l += [[name,e] for e in set(dic[name])]

    for row in l:
        row.append(sum(value == row[1] for value in dic[row[0]]))

    df_w = pd.DataFrame(l, columns=['source', 'target', 'weight'])
    df_w = df_w[df_w['source'] != df_w['target']]
    
    
    return df_w.values.tolist()

def create_data(interval):
    filepath_main = '../data/'
    df = pd.read_csv(filepath_main + 'email headers.csv', encoding='cp1252')
    df.Date = pd.to_datetime(df.Date)
    df.head()

    dffrom = list(df.From)
    dfto = list(df.To)
    dftone = []
    for row in dfto:
        dftone.append(row.split(', '))

    dffrom2 = []
    for row in dffrom:
        if row.split('@')[1] == "gastech.com.tethys":
            dffrom2.append((row.split('@')[0].split('.')[0] + ' ' + row.split('@')[0].split('.')[1]) + ' (tethys)')
        else:
            dffrom2.append((row.split('@')[0].split('.')[0] + ' ' + row.split('@')[0].split('.')[1]))

    ef_to = []
    for row in dftone:
        l = []
        for name in row:
            if name.split('@')[1] == "gastech.com.tethys":
                l.append((name.split('@')[0].split('.')[0] + ' ' + name.split('@')[0].split('.')[1]) + ' (tethys)')
            l.append((name.split('@')[0].split('.')[0] + ' ' + name.split('@')[0].split('.')[1]))
        ef_to.append(l)
    df = pd.DataFrame({ 'from': dffrom2, 'to':ef_to, 'date': df.Date, 'subject':df.Subject})
    
    df['interval'] = df['date'].dt.round('{}min'.format((str(interval))))


    data = df.values.tolist()
    dicdat = {}
    for row in data:
        day = str(row[4])
        if day in dicdat.keys():
            dicdat[day].append(row)
        else:
            dicdat[day] = [row]
                    
    l = []
    
    for key in dicdat.keys():
        l.append([key, (create_networkformat_perday(key, dicdat))])

    pd.DataFrame(l, columns=['Day', 'Network']).to_csv('../data/networkplot_data/network_data_{}.csv'.format(str(interval)))


for inter in [5,10,15,20,30,60,120]:
    create_data(inter)
    create_data_subjects(inter)

In [4]:
def create_networkformat_perday(day:str, dicdat):
    data = dicdat[day]
    df = pd.DataFrame(data, columns = ['from','to','date','subject', 'day'])
    unique_froms = (set(list(df['from'])))
    
    dic = {}
    for i in range(len(df['from'])):
        k = df['from'][i]
        if k in dic.keys():
            dic[k] += df['to'][i]
        else:
            dic[k] = df['to'][i]

    l = []
    for name in unique_froms:
        l += [[name,e] for e in set(dic[name])]

    for row in l:
        row.append(sum(value == row[1] for value in dic[row[0]]))

    df_w = pd.DataFrame(l, columns=['source', 'target', 'weight'])
    df_w = df_w[df_w['source'] != df_w['target']]
    
    
    return df_w.values.tolist()

def create_data(interval):
    filepath_main = '../data/'
    df = pd.read_csv(filepath_main + 'email headers.csv', encoding='cp1252')
    df.Date = pd.to_datetime(df.Date)
    df.head()

    dffrom = list(df.From)
    dfto = list(df.To)
    dftone = []
    for row in dfto:
        dftone.append(row.split(', '))

    dffrom2 = []
    for row in dffrom:
        if row.split('@')[1] == "gastech.com.tethys":
            dffrom2.append((row.split('@')[0].split('.')[0] + ' ' + row.split('@')[0].split('.')[1]) + ' (tethys)')
        else:
            dffrom2.append((row.split('@')[0].split('.')[0] + ' ' + row.split('@')[0].split('.')[1]))

    ef_to = []
    for row in dftone:
        l = []
        for name in row:
            if name.split('@')[1] == "gastech.com.tethys":
                l.append((name.split('@')[0].split('.')[0] + ' ' + name.split('@')[0].split('.')[1]) + ' (tethys)')
            l.append((name.split('@')[0].split('.')[0] + ' ' + name.split('@')[0].split('.')[1]))
        ef_to.append(l)
    df = pd.DataFrame({ 'from': dffrom2, 'to':ef_to, 'date': df.Date, 'subject':df.Subject})
    
    df['interval'] = df['date'].dt.round('{}min'.format((str(interval))))


    data = df.values.tolist()
    dicdat = {}
    for row in data:
        day = str(row[4])
        if day in dicdat.keys():
            dicdat[day].append(row)
        else:
            dicdat[day] = [row]
                    
    l = []
    
    for key in dicdat.keys():
        l.append([key, (create_networkformat_perday(key, dicdat))])

    pd.DataFrame(l, columns=['Day', 'Network']).to_csv('../data/networkplot_data/network_data_{}.csv'.format(str(interval)))


In [5]:
for inter in [5,10,15,20,30,60,120]:
    create_data(inter)

In [7]:
df = pd.read_csv('../data/email headers.csv', encoding='cp1252')
df

,From,To,Date,Subject
0,Sven.Flecha@gastech.com.kronos,"Isak.Baza@gastech.com.kronos, Lucas.Alcazar@ga...",1/6/2014 8:39,GT-SeismicProcessorPro Bug Report
1,Kanon.Herrero@gastech.com.kronos,"Felix.Resumir@gastech.com.kronos, Hideki.Cocin...",1/6/2014 8:58,Inspection request for site
2,Bertrand.Ovan@gastech.com.kronos,"Emile.Arpa@gastech.com.kronos, Varro.Awelon@ga...",1/6/2014 9:28,New refueling policies - Effective February 1
3,Valeria.Morlun@gastech.com.kronos,"Dante.Coginian@gastech.com.kronos, Albina.Hafo...",1/6/2014 9:38,Route suggestion for next shift
4,Mat.Bramar@gastech.com.kronos,"Rachel.Pantanal@gastech.com.kronos, Lars.Azada...",1/6/2014 9:49,Upcoming birthdays
...,...,...,...,...
1165,Mat.Bramar@gastech.com.kronos,"Mat.Bramar@gastech.com.kronos, Anda.Ribera@gas...",1/17/2014 19:47,RE: Training opportunity
1166,Sven.Flecha@gastech.com.kronos,"Nils.Calixto@gastech.com.kronos, Sven.Flecha@g...",1/17/2014 19:56,RE: The most creative spam I've seen lately.
1167,Dylan.Scozzese@gastech.com.kronos,"Albina.Hafon@gastech.com.kronos, Valeria.Morlu...",1/17/2014 20:04,RE: Traffic advisory for today
1168,Varja.Lagos@gastech.com.kronos,"Hideki.Cocinaro@gastech.com.kronos, Varja.Lago...",1/17/2014 20:15,RE: Inspection request for site
